<a href="https://colab.research.google.com/github/whoami02/ML/blob/master/LLM%20on%20HP%20book.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q python-dotenv langchain transformers PyPDF2 weaviate-client sentence-transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.6/808.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.2/188.2 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.1 MB/s eta 0:00:00


In [1]:
from dotenv import load_dotenv
# from langchain import OpenAI
from langchain import HuggingFaceHub, LLMChain

In [2]:
load_dotenv()

True

In [9]:
# llm = OpenAI(model='gpt-3.5-turbo-1106' ,temperature=2)
# tools = load_tools(['serpapi', 'llm-math'], llm=llm)

hub_llm = HuggingFaceHub(repo_id="tiiuae/falcon-7b-instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables = ["question"],
    template = "As a domain expert you have an unparalleled knowledge on the topic, you are the best in the buisness and your word is the law. As an expert, some poor individuals seek some percentage of your greatness to succeed. You will guide and help them excel by answering their queries :{question}"
    # template = "wkwkwkw: {question}"
)

In [ ]:
hub_chain = LLMChain(prompt=prompt, llm=hub_llm, verbose=True)

In [ ]:
print(hub_chain.run("What are the steps to deply a machine learning application?"))



> Entering new LLMChain chain...
Prompt after formatting:
As a domain expert you have an unparalleled knowledge on the topic, you are the best in the buisness and your word is the law. As an expert, some poor individuals seek some percentage of your greatness to succeed. You will guide and help them excel by answering their queries :What are the steps to deply a machine learning application?

> Finished chain.

1. Define the problem: Understand the problem and its requirements.
2. Choose the right tools: Select the right tools and frameworks for the job.
3. Gather data: Collect and process data to train the machine learning model.
4. Train the model: Train the model using the data and algorithms.
5. Evaluate the model: Evaluate the model's performance and refine it if necessary.
6. Deploy the model: Deploy the model in production and monitor its performance.
7. Maintain the model: Keep the model up to date with new data and trends.


In [ ]:
print(hub_chain.run("what are the steps to deploy a machine learning application"))



> Entering new LLMChain chain...
Prompt after formatting:
As a domain expert you have an unparalleled knowledge on the topic, you are the best in the buisness and your word is the law. As an expert, some poor individuals seek some percentage of your greatness to succeed. You will guide and help them excel by answering their queries :what are the steps to deploy a machine learning application

> Finished chain.
?
1. Define the problem: Understand the problem and its requirements.
2. Choose a framework: Select a suitable machine learning framework.
3. Data collection: Collect and preprocess the data.
4. Feature engineering: Select and clean the features.
5. Model selection: Select the appropriate model.
6. Model training: Train the model.
7. Model evaluation: Evaluate the model.
8. Model deployment: Deploy the model.
9. Model maintenance: Monitor the model.
10. Model adaptation: Adapt the model to changing conditions.


In [3]:
from PyPDF2 import  PdfReader
from langchain.vectorstores import Weaviate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain
from tqdm import tqdm
import re

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)
import torch

In [4]:
pdf_reader = PdfReader(r"/content/HP-6--Harry-Potter-and-the-Half-Blood-Prince.pdf")
book_ = pdf_reader.pages[16:]

In [5]:
text = []
for i in tqdm(range(0,len(pdf_reader.pages)-16)):
    text.append(book_[i].extract_text())


100%|██████████| 655/655 [00:09<00:00, 72.37it/s]


In [6]:
corpus = ''
for i in tqdm(range(len(text))):
    temp = text[i]
    temp = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", temp)
    temp = re.sub(r" +", " ", temp)
    temp = re.sub('C H A P T E R ', '', temp)
    corpus += temp

100%|██████████| 655/655 [00:00<00:00, 3225.88it/s]


In [22]:
corpus[:500]

'THE OTHER MINISTER 3 the dark glass He knew that cough He had heard it before He turned very slowly to face the empty room Hello he said trying to sound braver than he felt For a brief moment he allowed hi mself the impossible hope that nobody would answer him However a voice responded at once a crisp decisive voice that sounded as though it were reading a prepared statement It was coming as the Prime Minister had known at the first cough from th e froglike little man wearing a long silver wig w'

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap  = 32,
    length_function = len,
)
texts = text_splitter.split_text(corpus)
texts = text_splitter.create_documents(texts)

In [9]:
import weaviate

auth_config = weaviate.AuthApiKey(api_key="RPlCu70oXcOsVsWUWf0sjMNlrV5xFU7oatgA")

client = weaviate.Client(
  url="https://idk-915fsd3w.weaviate.network",
  auth_client_secret=auth_config
)


In [10]:
# embeddings = "sentence-transformers/all-mpnet-base-v2"
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name,
  model_kwargs=model_kwargs
)

In [11]:
docsearch = Weaviate.from_documents(texts, embeddings, client=client, by_text=False)

In [12]:
docsearch.similarity_search("tell about horace slughorn")

[Document(page_content='isnt it said Slughorn HORACE SLUGHORN 71 Not really sa id Harry coldly Slughorn looked down at him in surprise You mustnt think Im prejudiced he said No no no Havent I just said your mother was one of my alltime favorite students And there was Dirk Cresswell in the year afte r her too now Head of the Goblin Liaison Office of course another Muggleborn a very gifted student and still gives me excell ent inside information on the goingson at Gringotts He bounced up and down a little smiling in a selfsatisfied'),
 Document(page_content='Slughorn As they set off down the garden pa th Slughorns voice floated after them Ill want a pay rise Dumbledore Dumbledore chuckled The garden gate swung shut behind them and they set off back down th e hill through the dark and the swirling mist Well done Harry said Dumbledore I didnt do anything said Harry in surprise Oh yes you did You showed Horace exactly how much he stands to gain by returning to Hogwarts Did you like him Er H

In [13]:
model = "tiiuae/falcon-7b-instruct" #tiiuae/falcon-40b-instruct

tokenizer = AutoTokenizer.from_pretrained(model, return_lenght=False, return_token_type_ids=False)

model = AutoModelForCausalLM.from_pretrained(
    model,
    torch_dtype = torch.bfloat16,
    # quantization_config = BitsAndBytesConfig(
        # load_in_8bit = True,
        # bnb_in_8bit_use_double_quant=True,
        # bnb_8bit_quant_type = "nf4",
        # load_in_8bit_fp32_cpu_offload=True,
    # ),
    device_map="auto",
    offload_folder="offload",
    trust_remote_code=True
)
pipeline = pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    # offload_folder="./offload",
    device_map="auto",
    max_length=2056,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
from langchain.prompts import PromptTemplate

prompt_template = """ Use the following pieces of context to answer the questions at the end. If you don't know the answer, please think rationally and provide the relevant answer from your knowledge base,
{context}
Question : {question}
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables = ["context", "question"]
)

In [15]:
llm = HuggingFacePipeline(pipeline = pipeline)

In [16]:
chain_type_kwargs = {"prompt":PROMPT}
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(), chain_type_kwargs=chain_type_kwargs
)

In [21]:
response = qa_chain.run("Who does Hagrid bury in his backyard?")
print(response)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Answer: Hagrid buries Aragog in his backyard in Deathly Hallows.
